In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os
import copy
import time
import pprint
import shutil
from scipy.optimize import linprog
from pytest import approx
from IPython.display import HTML, display

## Creando archivos `.pyx`

In [2]:
os.chdir("..")
shutil.copyfile('src/mex/simplex/simplex_networks.py','notebooks/reimplementacion_0/simplex_networks_0.pyx')
shutil.copyfile('src/mex/simplex/problem_definition.py','notebooks/reimplementacion_0/problem_definition_0.pyx')
shutil.copyfile('src/mex/utils/general.py','notebooks/reimplementacion_0/general_0.pyx')
os.chdir("notebooks")

## Creando archivos `setup.py`

In [4]:
%%file reimplementacion_0/setup_simplexnetworks_0.py
from distutils.core import setup
from Cython.Build import cythonize
setup(ext_modules = cythonize("reimplementacion_0/simplex_networks_0.pyx", compiler_directives={'language_level' : 3}))

Overwriting reimplementacion_0/setup_simplexnetworks_0.py


In [5]:
%%file reimplementacion_0/setup_problemdefinition_0.py
from distutils.core import setup
from Cython.Build import cythonize
setup(ext_modules = cythonize("reimplementacion_0/problem_definition_0.pyx", compiler_directives={'language_level' : 3}))

Overwriting reimplementacion_0/setup_problemdefinition_0.py


In [6]:
%%file reimplementacion_0/setup_general_0.py
from distutils.core import setup
from Cython.Build import cythonize
setup(ext_modules = cythonize("reimplementacion_0/general_0.pyx", compiler_directives={'language_level' : 3}))

Overwriting reimplementacion_0/setup_general_0.py


## Compilando

In [7]:
%%bash
python3 reimplementacion_0/setup_simplexnetworks_0.py build_ext --inplace

Compiling reimplementacion_0/simplex_networks_0.pyx because it changed.
[1/1] Cythonizing reimplementacion_0/simplex_networks_0.pyx
running build_ext
building 'simplex_networks_0' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/reimplementacion_0
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c reimplementacion_0/simplex_networks_0.c -o build/temp.linux-x86_64-3.7/reimplementacion_0/simplex_networks_0.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.7/reimplementacion_0/simplex_networks_0.o -o /datos/practica-2-segunda-parte-caroacostatovany/notebooks/simplex_networks_0.cpython-37m-x86_64-linux-gnu.so


In [8]:
%%bash
python3 reimplementacion_0/setup_problemdefinition_0.py build_ext --inplace

Compiling reimplementacion_0/problem_definition_0.pyx because it changed.
[1/1] Cythonizing reimplementacion_0/problem_definition_0.pyx
running build_ext
building 'problem_definition_0' extension
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c reimplementacion_0/problem_definition_0.c -o build/temp.linux-x86_64-3.7/reimplementacion_0/problem_definition_0.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.7/reimplementacion_0/problem_definition_0.o -o /datos/practica-2-segunda-parte-caroacostatovany/notebooks/problem_definition_0.cpython-37m-x86_64-linux-gnu.so


In [9]:
%%bash
python3 reimplementacion_0/setup_general_0.py build_ext --inplace

Compiling reimplementacion_0/general_0.pyx because it changed.
[1/1] Cythonizing reimplementacion_0/general_0.pyx
running build_ext
building 'general_0' extension
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c reimplementacion_0/general_0.c -o build/temp.linux-x86_64-3.7/reimplementacion_0/general_0.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.7/reimplementacion_0/general_0.o -o /datos/practica-2-segunda-parte-caroacostatovany/notebooks/general_0.cpython-37m-x86_64-linux-gnu.so


### Objetivo

In [11]:
c_max_obj_1 = [-3, -5]
A_max_obj_1 = [[1, 0], [0, 2], [3, 2]]
b_max_obj_1 = [4, 12, 18]

In [12]:
max_obj_1 = -1*linprog(c_max_obj_1, A_ub=A_max_obj_1, b_ub=b_max_obj_1).fun

In [13]:
coeff_obj_1 = linprog(c_max_obj_1, A_ub=A_max_obj_1, b_ub=b_max_obj_1).x

### Aproximación

In [14]:
from simplex_networks_0 import create_matrix, pivots_col, pivots_row, find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot
from problem_definition_0 import add_cons, constrain, add_obj, obj, maxz, minz

In [15]:
n_var_approx_1 = 2
n_cons_approx_1 = 3

In [16]:
# Todo junto
start_time = time.time()
matrix_max_approx_1 = create_matrix(n_var_approx_1,n_cons_approx_1)
constrain(matrix_max_approx_1,'1,0,L,4')
constrain(matrix_max_approx_1,'0,2,L,12')
constrain(matrix_max_approx_1,'3,2,L,18')
obj(matrix_max_approx_1,'3,5,0')
problem_approx_1 = maxz(matrix_max_approx_1)
max_approx_1 = problem_approx_1['max']
problem_approx_1.pop('max')
coeff_approx_1 = np.array(list(problem_approx_1.values()))
end_time = time.time()
secs = end_time-start_time
print("Todo el proceso tomó",secs,"segundos" )

Todo el proceso tomó 0.0019145011901855469 segundos


### Comprobación

In [17]:
assert max_obj_1 == approx(max_approx_1), "El valor aproximado es incorrecto"
assert coeff_obj_1 == approx(coeff_approx_1), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", max_obj_1)
print("El valor aproximado obtenido con mex es: ", max_approx_1)
print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj_1)
print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx_1)

El valor objetivo obtenido con scipy es:  35.99999997843227
El valor aproximado obtenido con mex es:  36.0
Los coeficientes objetivos obtenidos con scipy son:  [2. 6.]
Los coeficientes aproximados obtenidos con mex son:  [2. 6.]


## Anotaciones para analizar bloques de código

In [18]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate reimplementacion_0/simplex_networks_0.pyx
$HOME/.local/bin/cython --force -3 --annotate reimplementacion_0/problem_definition_0.pyx
$HOME/.local/bin/cython --force -3 --annotate reimplementacion_0/general_0.pyx

In [20]:
display(HTML("reimplementacion_0/simplex_networks_0.html"))

In [21]:
display(HTML("reimplementacion_0/problem_definition_0.html"))

In [22]:
display(HTML("reimplementacion_0/general_0.html"))